In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [1]:
%cd '/content/drive/My Drive/Workspace Riset Tata/thesis'

/content/drive/My Drive/Workspace Riset Tata/thesis


In [0]:
%cd '/content/drive/My Drive/Workspace Riset Tata/thesis'
import config_16x15_seq
%cd '/content/drive/My Drive/Workspace Riset Tata/thesis/config_16x15_seq'

/content/drive/My Drive/thesis
/content/drive/My Drive/thesis/config_16x15_seq


In [2]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
    device_name = tf.test.gpu_device_name()
    if device_name != '/device:GPU:0':
        print('ERROR!')
    else:
        print('Found GPU at: {}'.format(device_name))
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print('TPU address is', tpu_address)
    with tf.Session(tpu_address) as session:
        devices = session.list_devices()
        print('TPU devices:')
        pprint.pprint(devices)



Found GPU at: /device:GPU:0


In [0]:
from config_16x15_seq.builder import *
from tqdm import tqdm
from keras.callbacks import BaseLogger, History, CallbackList
from scipy.ndimage import gaussian_filter
import copy

MU_training = MU['training']
MU_norm_training = MU_norm['training']
PHI_meas_training = PHI_meas['training']
MUa_training = MUa['training']
MUsp_training = MUsp['training']
freq_training = freq['training']
d_training = d['training']
lr = 0.0002
beta_1 = 0.5
# clip_value = 0.01
optimizer = Adam(lr=lr, beta_1=beta_1)
# optimizer = RMSprop(lr=lr)
generator = primary_net()
print('Generator model summary:')
generator.summary()
discriminator = secondary_net()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[custom_binary_accuracy])
print('Discriminator model summary:')
discriminator.summary()
discriminator.trainable = False
# inputs = Input((PHI_meas_training.shape[1],))
outputs = generator.outputs
outputs.append(discriminator(outputs[:2]))
gan = Model(inputs=generator.inputs, outputs=outputs)
loss = ['mse', 'mse', 'mse', 'mse', 'mse', 'mse', 'binary_crossentropy']
loss_weights = [0, 0, 5, 5, 1e4, 1, 1]
metrics = ['mae', 'mse', 'mape', 'msle', 'logcosh', 'cosine', custom_binary_accuracy]
gan.compile(loss=loss, loss_weights=loss_weights, optimizer=optimizer, metrics=metrics)
print('Generative adversarial network model summary:')
gan.summary()
history = History()
logger = BaseLogger(stateful_metrics=['discriminator_' + s for s in discriminator.stateful_metric_names] +
                                     ['gan_' + s for s in gan.stateful_metric_names])
# checkpoint = callbacks.ModelCheckpoint('model_test{epoch:02d}.h5', verbose=1)
time_history = TimeHistory()
callbacks = [logger, time_history, history]
out_labels = ['discriminator_' + s for s in discriminator.metrics_names] + ['gan_' + s for s in gan.metrics_names]
callback_metrics = copy.copy(out_labels) + ['val_' + n for n in out_labels]
callbacks = CallbackList(callbacks)
# callbacks = [TimeHistory(), EarlyStopping(monitor='loss', min_delta=0.1, patience=100,
#                                           restore_best_weights=True, verbose=1)]
# early_stop = EarlyStopping(patience=200, verbose=1)
choice = np.zeros(data_size, dtype='uint8')
choice[np.random.choice(data_size, val_size, replace=False)] = 1
MU_training_train = MU_training[choice == 0]
MU_training_val = MU_training[choice == 1]
MU_norm_training_train = MU_norm_training[choice == 0]
MU_norm_training_val = MU_norm_training[choice == 1]
PHI_meas_training_train = PHI_meas_training[choice == 0]
PHI_meas_training_val = PHI_meas_training[choice == 1]
freq_training_train = freq_training[choice == 0]
freq_training_val = freq_training[choice == 1]
d_training_train = d_training[choice == 0]
d_training_val = d_training[choice == 1]
MUa_train = MUa_training[choice == 0]
MUa_val = MUa_training[choice == 1]
MUsp_train = MUsp_training[choice == 0]
MUsp_val = MUsp_training[choice == 1]
# choice = data_cat
batch_size = 32
prob_param = 20.0
prob_param2 = 1.0
# label_softness = 0.1
# metrics_eval_n = 50
# metrics_eval_step = 20
hard_valid = np.ones(val_size)
hard_fake = np.zeros(val_size)


# soft_label = False
# augment = True
# min_img_loss_avg = np.inf
# min_dis_loss_avg = np.inf
# img_loss_baseline_factor = 1.1
# img_loss_threshold = 1.2
# img_loss_threshold2 = 0.6
# prob_augment = 0.5
# gan_d_acc = [0.5, 0.5]
# init_train = 100

N_count_train = np.zeros_like(N_count)
n = 0
for i in range(len(N_count)):
    ni = N_count[i]
    N_count_train[i] = np.where(choice[n:(n + ni)] == 0)[0].size
    n += ni
index_array = np.arange(train_size)
index_array2 = np.arange(train_size)
ia = np.arange(train_size)
n = N_count_train[0]
np.random.shuffle(index_array[:n])
np.random.shuffle(index_array2[:n])
for i in range(1, len(N_count_train) - 1):
    ni = N_count_train[i]
    np.random.shuffle(index_array[n:(n + ni)])
    np.random.shuffle(index_array2[n:(n + ni)])
    n += ni
np.random.shuffle(index_array[n:])
np.random.shuffle(index_array2[n:])


def run_epoch(epochs):
    callbacks.set_params({
        'batch_size': batch_size,
        'epochs': epochs,
        'steps': None,
        'samples': train_size,
        'verbose': 2,
        'do_validation': True,
        'metrics': callback_metrics,
    })
    callbacks.on_train_begin()
    for epoch in range(epochs):
        for m in discriminator.stateful_metric_functions:
            m.reset_states()
        for m in gan.stateful_metric_functions:
            m.reset_states()
        callbacks.on_epoch_begin(epoch)
        epoch_logs = {}
        progress_bar = None
        # np.random.shuffle(ia)
        n = N_count_train[0]
        np.random.shuffle(ia[:n])
        for i in range(1, len(N_count_train) - 1):
            ni = N_count_train[i]
            np.random.shuffle(ia[n:(n + ni)])
            n += ni
        np.random.shuffle(ia[n:])
        num_batches = (train_size + batch_size - 1) // batch_size
        # if epoch > 0 and epoch % metrics_eval_step == 0:
        # hist_avg = {}
        # for k, v in history.history.items():
        #     if k.startswith('val_'):
        #         l = k[4:]
        #     else:
        #         l = k
        #     if l in logger.stateful_metrics:
        #         hist_avg[k] = v[-1]
        #     else:
        #         hist_avg[k] = np.average(v[-metrics_eval_n:])
        # img_loss_avg = hist_avg['val_gan_' + gan.metrics_names[0]] - hist_avg['val_gan_' + gan.metrics_names[7]]
        # dis_loss_avg = hist_avg['val_discriminator_' + discriminator.metrics_names[0]]
        # min_img_loss = np.min(history.history['val_gan_' + gan.metrics_names[0]]) \
        #                - np.min(history.history['val_gan_' + gan.metrics_names[7]])
        # if img_loss_avg > min_img_loss_avg * img_loss_baseline_factor:
        #     if not augment:
        #         augment = True
        #     elif img_loss_avg >= img_loss_threshold and min_img_loss >= img_loss_threshold2:
        #         soft_label = True
        # elif dis_loss_avg > min_dis_loss_avg:
        #     if augment:
        #         augment = False
        #     elif img_loss_avg < img_loss_threshold:
        #         soft_label = False
        # img_loss_avg = np.average(history.history['val_gan_' + gan.metrics_names[0]][-metrics_eval_n:])\
        #                - np.average(history.history['val_gan_' + gan.metrics_names[7]][-metrics_eval_n:])
        # if soft_label and img_loss_avg < img_loss_threshold:
        #     soft_label = False
        # if np.random.random() > 0.5:
        #     augment = not augment
        # min_img_loss_avg = min(img_loss_avg, min_img_loss_avg)
        # min_dis_loss_avg = min(dis_loss_avg, min_dis_loss_avg)
        # if epoch >= init_train:
        #     if gan_d_acc[0] < 0.5 or gan_d_acc[1] < 0.5:
        #         prob_augment = 0.8
        #     else:
        #         prob_augment = 0.5
        #     if augment == (np.random.random() < prob_augment):
        #         augment = not augment
        batch_end = 0
        # prob_augment = np.random.random()
        for batch_index in range(num_batches):
            augment = np.random.random() < 0.5
            batch_start = batch_end
            batch_end = min(train_size, batch_end + batch_size)
            batch_ids = index_array[ia[batch_start:batch_end]]
            batch_logs = {'batch': batch_index, 'size': len(batch_ids)}
            callbacks.on_batch_begin(batch_index, batch_logs)
            MU_gt = [MU_training_train[batch_ids, 0], MU_training_train[batch_ids, 1]]
            MU_gt = [im.reshape(im.shape[:1] + (1,) + im.shape[1:]) for im in MU_gt]
            in_gen = [PHI_meas_training_train[batch_ids], freq_training_train[batch_ids], d_training_train[batch_ids]]
            if augment:
                noise_param = np.random.random(len(batch_ids))
                blur_param = np.random.random(len(batch_ids))
                noise_param *= blur_param
                MU_gt_aug = [np.copy(im) for im in MU_gt]
                for i, bi in enumerate(batch_ids):
                    temp = mask_image(MU_gt_aug[0][i, 0], MUa_train[bi])
                    temp = temp + temp * np.random.normal(scale=noise_param[i] * 0.5, size=temp.shape)
                    temp = gaussian_filter(temp, blur_param[i] * 5.0)
                    MU_gt_aug[0][i, 0] = mask_image(temp, 0.0)
                    temp = mask_image(MU_gt_aug[1][i, 0], MUsp_train[bi])
                    temp = temp + temp * np.random.normal(scale=noise_param[i] * 0.5, size=temp.shape)
                    temp = gaussian_filter(temp, blur_param[i] * 5.0)
                    MU_gt_aug[1][i, 0] = mask_image(temp, 0.0)
                # temp = np.clip(blur_param, prob_param2 / prob_param, 1.0 - prob_param2 / prob_param)
                valid = 0.95 - 0.05 * np.random.random(len(batch_ids))
                # valid = 1.0 - label_softness * np.random.beta(temp * prob_param, (1.0 - temp) * prob_param)
                # if soft_label:
                #     fake = label_softness * np.random.random(len(batch_ids))
                #     valid = 1.0 - label_softness * np.random.beta(temp * prob_param, (1.0 - temp) * prob_param)
                # else:
                #     fake = hard_fake[:len(batch_ids)]
                #     valid = hard_valid[:len(batch_ids)] - label_softness
            else:
                MU_gt_aug = MU_gt
                valid = hard_valid[:len(batch_ids)] - 0.05
                # valid = 1.0 - 0.05 * np.random.random(len(batch_ids))
                # valid = 1.0 - label_softness * np.random.random(len(batch_ids))
                # valid = 1.0 - label_softness * np.random.beta(prob_param2,
                #                                               prob_param - prob_param2, size=len(batch_ids))
                # if soft_label:
                #     fake = label_softness * np.random.random(len(batch_ids))
                #     valid = 1.0 - label_softness * np.random.beta(
                #         prob_param2, prob_param - prob_param2, size=len(batch_ids))
                # else:
                #     fake = hard_fake[:len(batch_ids)]
                #     valid = hard_valid[:len(batch_ids)] - label_softness
            # fake = hard_fake[:len(batch_ids)]
            fake = 0.05 * np.random.random(len(batch_ids))
            # if epoch < init_train:
            #     valid -= label_softness * np.random.random(len(batch_ids)) * (init_train - epoch) / init_train
            #     fake += label_softness * np.random.random(len(batch_ids)) * (init_train - epoch) / init_train
            d_loss1 = discriminator.train_on_batch(MU_gt_aug, valid)
            # d_loss1 = discriminator.train_on_batch([np.concatenate([im1, im2], axis=0)
            #                                        for im1, im2 in zip(MU_gt_aug, MU_gt)],
            #                                        np.concatenate([valid, hard_valid[:len(batch_ids)]], axis=0),
            #                                        sample_weight=np.ones(2 * len(batch_ids)) * 0.5)
            d_loss2 = discriminator.train_on_batch(generator.predict(in_gen)[:2], fake)
            # d_loss = discriminator.train_on_batch([np.concatenate([im1, im2], axis=0)
            #                                        for im1, im2 in zip(MU_gt_aug, generator.predict(in_gen)[:2])],
            #                                       np.concatenate([valid, fake], axis=0))
            d_loss = np.zeros(len(discriminator.metrics_names))
            for i, l in enumerate(discriminator.metrics_names):
                if l in discriminator.stateful_metric_names:
                    d_loss[i] = d_loss2[i]
                else:
                    d_loss[i] = 0.5 * (d_loss1[i] + d_loss2[i])
            # for l in discriminator.layers:
            #     weights = l.get_weights()
            #     weights = [np.clip(w, -clip_value, clip_value) for w in weights]
            #     l.set_weights(weights)
            batch_ids = index_array2[ia[batch_start:batch_end]]
            MU_gt = [MU_training_train[batch_ids, 0], MU_training_train[batch_ids, 1]]
            MU_gt = [im.reshape(im.shape[:1] + (1,) + im.shape[1:]) for im in MU_gt]
            in_gen = [PHI_meas_training_train[batch_ids], freq_training_train[batch_ids], d_training_train[batch_ids]]
            MU_norm_gt = [MU_norm_training_train[batch_ids, 0], MU_norm_training_train[batch_ids, 1]]
            MU_norm_gt = [im.reshape(im.shape[:1] + (1,) + im.shape[1:]) for im in MU_norm_gt]
            g_loss = gan.train_on_batch(in_gen, [MU_gt[0], MU_gt[1], MU_norm_gt[0], MU_norm_gt[1],
                                                 MUa_train[batch_ids], MUsp_train[batch_ids],
                                                 hard_valid[:len(batch_ids)]])
            for l, o in zip(out_labels, np.concatenate([d_loss, g_loss])):
                batch_logs[l] = o
            callbacks.on_batch_end(batch_index, batch_logs)
            if progress_bar is None:
                progress_bar = tqdm(total=train_size, desc='Epoch {}/{}'.format(epoch + 1, epochs), unit='samples')
            progress_bar.set_postfix(train_on_batch=('[D loss: [%g %g], acc.: [%.2f%% %.2f%%]] [G total_loss: %g, ' +
                                                     'img_loss: %g, d_loss: %g, d_acc.: %.2f%%]') %
                                                    (d_loss1[0], d_loss2[0], 100 * d_loss1[1], 100 * d_loss2[1],
                                                     g_loss[0],
                                                     g_loss[0] - g_loss[7], g_loss[7], 100 * g_loss[-1]),
                                     augment=augment)
            progress_bar.update(len(batch_ids))
        MU_gt = [MU_training_val[:, np.array([0])], MU_training_val[:, np.array([1])]]
        d_val_loss1 = discriminator.evaluate(MU_gt, hard_valid, batch_size=batch_size, verbose=0)
        d_val_loss2 = discriminator.evaluate(generator.predict([PHI_meas_training_val, freq_training_val,
                                                                d_training_val])[:2], hard_fake, batch_size=batch_size,
                                             verbose=0)
        # d_val_loss = discriminator.evaluate([np.concatenate([im1, im2], axis=0)
        #                                      for im1, im2 in zip(MU_gt,
        #                                                          generator.predict(PHI_meas_training_val)[:2])],
        #                                     np.concatenate([hard_valid, hard_fake], axis=0), verbose=0)
        d_val_loss = np.zeros(len(discriminator.metrics_names))
        for i, l in enumerate(discriminator.metrics_names):
            if l in discriminator.stateful_metric_names:
                d_val_loss[i] = d_val_loss2[i]
            else:
                d_val_loss[i] = 0.5 * (d_val_loss1[i] + d_val_loss2[i])
        MU_norm_gt = [MU_norm_training_val[:, np.array([0])], MU_norm_training_val[:, np.array([1])]]
        progress_bar.set_postfix(validate_on_epoch='[D loss: [%g %g], acc.: [%.2f%% %.2f%%]] validating generator..' %
                                                   (d_val_loss1[0], d_val_loss2[0], 100 * d_val_loss1[1],
                                                    100 * d_val_loss2[1]))
        g_val_loss = gan.evaluate([PHI_meas_training_val, freq_training_val, d_training_val],
                                  [MU_gt[0], MU_gt[1], MU_norm_gt[0], MU_norm_gt[1], MUa_val, MUsp_val, hard_valid],
                                  batch_size=batch_size, verbose=0)
        for l, o in zip(out_labels, np.concatenate([d_val_loss, g_val_loss])):
            epoch_logs['val_' + l] = o
        # num_batches = (val_size + batch_size - 1) // batch_size
        # for batch_index in range(num_batches):
        #     batch_start = batch_index * batch_size
        #     batch_end = min(val_size, (batch_index + 1) * batch_size)
        #     MU_gt = [MU_training_val[batch_start:batch_end, 0], MU_training_val[batch_start:batch_end, 0]]
        #     MU_gt = [im.reshape(im.shape[:1] + (1,) + im.shape[1:]) for im in MU_gt]
        #     d_loss1 = discriminator.test_on_batch(MU_gt, hard_valid[:(batch_end - batch_start)])
        #     in_gen = PHI_meas_training_val[batch_start:batch_end]
        #     d_loss2 = discriminator.test_on_batch(generator.predict(in_gen)[:2], hard_fake[:(batch_end - batch_start)])
        #     d_loss = np.zeros(len(discriminator.metrics_names))
        #     for i, l in enumerate(discriminator.metrics_names):
        #         if l in discriminator.stateful_metric_names:
        #             d_loss[i] = d_loss2[i]
        #         else:
        #             d_loss[i] = 0.5 * (d_loss1[i] + d_loss2[i])
        #     MU_norm_gt = [MU_norm_training_val[batch_start:batch_end, 0],
        #                   MU_norm_training_val[batch_start:batch_end, 1]]
        #     MU_norm_gt = [im.reshape(im.shape[:1] + (1,) + im.shape[1:]) for im in MU_norm_gt]
        #     g_loss = gan.test_on_batch(in_gen, [MU_gt[0], MU_gt[1], MU_norm_gt[0], MU_norm_gt[1],
        #                                       MUa_val[batch_start:batch_end], MUsp_val[batch_start:batch_end],
        #                                       hard_valid[:(batch_end - batch_start)]])
        #     for l, o in zip(out_labels, np.concatenate([d_loss, g_loss])):
        #         if l in logger.stateful_metrics:
        #             epoch_logs['val_' + l] = o
        #         else:
        #             if 'val_' + l in epoch_logs:
        #                 epoch_logs['val_' + l] += o * (batch_end - batch_start)
        #             else:
        #                 epoch_logs['val_' + l] = o * (batch_end - batch_start)
        #     pbar.set_postfix(validate_on_batch=('[D loss: %g, acc.: %.2f%%] [G total_loss: %g, img_loss: %g, ' +
        #                                         'd_loss: %g, d_acc.: %.2f%%]') %
        #                                        (d_loss[0], 100 * d_loss[1], g_loss[0], g_loss[0] - g_loss[7],
        #                                         g_loss[7], 100 * g_loss[-1]), soft_label=soft_label, augment=augment)
        #     pbar.update(batch_end - batch_start)
        # for l in out_labels:
        #     if l not in logger.stateful_metrics:
        #         epoch_logs['val_' + l] /= val_size
        callbacks.on_epoch_end(epoch, epoch_logs)
        # gan_d_acc[0] = epoch_logs[out_labels[-1]]
        # gan_d_acc[1] = epoch_logs['val_' + out_labels[-1]]
        progress_bar.set_postfix(train=('[D loss: %g, acc.: %.2f%%] [G total_loss: %g, img_loss: %g, d_loss: %g, ' +
                                        'd_acc.: %.2f%%]') %
                                       (epoch_logs[out_labels[0]], 100 * epoch_logs[out_labels[1]],
                                        epoch_logs[out_labels[2]],
                                        epoch_logs[out_labels[2]] - epoch_logs[out_labels[9]],
                                        epoch_logs[out_labels[9]],
                                        100 * epoch_logs[out_labels[-1]]),
                                 validation=('[D loss: %g, acc.: %.2f%%] [G total_loss: %g, img_loss: %g, ' +
                                             'd_loss: %g, d_acc.: %.2f%%]') %
                                            (epoch_logs['val_' + out_labels[0]],
                                             100 * epoch_logs['val_' + out_labels[1]],
                                             epoch_logs['val_' + out_labels[2]],
                                             epoch_logs['val_' + out_labels[2]] - epoch_logs['val_' + out_labels[9]],
                                             epoch_logs['val_' + out_labels[9]],
                                             100 * epoch_logs['val_' + out_labels[-1]]))
        progress_bar.close()
    callbacks.on_train_end()
    hist_data = {
        'train_start': time_history.train_time_start,
        'train_time': time_history.train_time,
        'epoch_time': time_history.times,
        'history': history.history,
        'choice': np.where(choice == 1)[0].tolist()
    }
    return hist_data


hist_data = run_epoch(100)

gan.save('model46log_100.h5')
generator.save('model46log_100_gen.h5')
discriminator.save('model46log_100_dis.h5')
with open('model_hist_data46.0log.json', 'w') as f:
    json.dump(hist_data, f)


Using TensorFlow backend.







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





Generator model summary:
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 7)      0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, 6)      0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
_____________________________________________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
Epoch 1/100:   0%|          | 0/8000 [00:00<?, ?samples/s, augment=1, train_on_batch=[D loss: [0.695669 0.842798], acc.: [62.50% 28.12%]] [G total_loss: 8.04025, img_loss: 7.38172, d_loss: 0.658526, d_acc.: 62.50%]]/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
Epoch 100/100: 100%|██████████| 8000/8000 [03:31<00:00, 42.04samples/s, train=[D loss: 0.178141, acc.: 99.86%] [G total_loss: 2.61244, img_loss: 2.50075, d_loss: 0.111695, d_acc.: 98.95%], validation=[D loss

In [2]:
from builder import *
from tqdm import tqdm
from keras.callbacks import BaseLogger, History, CallbackList, ModelCheckpoint
from scipy.ndimage import gaussian_filter
from scipy.stats import truncnorm
from keras.preprocessing.image import ImageDataGenerator


# def blur_image(image):
#     blur_param = np.random.random()
#     noise_param = np.random.random() * blur_param
#     image = image + image * truncnorm.rvs(-2, 2, scale=noise_param * 0.5, size=image.shape)
#     image = gaussian_filter(image, blur_param * 5.0)
#     return image


def transform_theta(points, theta):
    theta = theta * (np.pi / 180)
    x = points[:, [0]] - 0.5
    y = -(points[:, [1]] - 0.5)
    x1 = x * np.cos(theta) - y * np.sin(theta) + 0.5
    y1 = -x * np.sin(theta) - y * np.cos(theta) + 0.5
    x = points[:, [2]] - 0.5
    y = -(points[:, [3]] - 0.5)
    x2 = x * np.cos(theta) - y * np.sin(theta) + 0.5
    y2 = -x * np.sin(theta) - y * np.cos(theta) + 0.5
    return np.concatenate([x1, y1, x2, y2], axis=1)


key = 'training_16_old'
MU_training = MU[key]
MU_norm_training = MU_norm[key]
PHI_meas_training = PHI_meas[key]
MUa_training = MUa[key]
MUsp_training = MUsp[key]
lr = 0.0002
beta_1 = 0.5
# clip_value = 0.01
# optimizer = Adam(lr=lr, beta_1=beta_1)
# optimizer = RMSprop(lr=lr)
generator, gen_names = primary_net()
print('Generator model summary:')
generator.summary()
discriminator, dis_names = secondary_net()
discriminator.name = 'discriminator'
metric_groups = [['mae', 'mse', 'mape', 'msle', 'logcosh', 'cosine'], [custom_binary_accuracy]]
# dis_losses = ['binary_crossentropy', 'mse', 'msle']
# dis_lw = [1, 20, 0.01]
# discriminator.compile(loss=dis_losses, loss_weights=dis_lw, optimizer=optimizer,
#                       metrics={dis_names[0]: metric_groups[1], dis_names[1]: metric_groups[0],
#                                dis_names[2]: metric_groups[0]})
dis_losses = ['binary_crossentropy']
dis_lw = [1]
discriminator.compile(loss=dis_losses, optimizer=Adam(lr=lr, beta_1=beta_1), metrics=metric_groups[1])
print('Discriminator model summary:')
discriminator.summary()
discriminator.trainable = False
# inputs = Input((None,)+PHI_meas_training.shape[2:])
# outputs = generator(inputs)
outputs = generator.outputs
if len(dis_names) == 1:
    outputs += [discriminator(outputs[:2])]
else:
    outputs += discriminator(outputs[:2])
gan = Model(inputs=generator.inputs, outputs=outputs)
losses = ['mse', 'mse', 'mse', 'mse', 'mse', 'mse'] + dis_losses
loss_weights = [0, 0, 5, 5, 1e4, 1] + dis_lw
metrics = {}
for name in gen_names:
    metrics[name] = metric_groups[0]
if len(dis_names) == 1:
    metrics[discriminator.name] = metric_groups[1]
else:
    metrics[discriminator.name] = metric_groups[1] + metric_groups[0]
dis_metrics_len = len(metrics[discriminator.name])
# gan_d_loss_idx_offset = 7 + len(gen_names) * len(metric_groups[0])
gan.compile(loss=losses, loss_weights=loss_weights, optimizer=Adam(lr=lr, beta_1=beta_1), metrics=metrics)
print('Generative adversarial network model summary:')
gan.summary()
dataset = datasets[key]
val_size = dataset.val_size
data_size = dataset.data_size
train_size = dataset.train_size
count_label = dataset.count_label
pos_data = dataset.pos_data
inc_areas = dataset.inc_areas
N_count = dataset.N_count
choice = np.zeros(data_size, dtype='uint8')
choice[np.random.choice(data_size, val_size, replace=False)] = 1
# choice = dataset.data_label
MU_training_train = MU_training[choice == 0]
MU_training_val = MU_training[choice == 1]
MU_norm_training_train = MU_norm_training[choice == 0]
MU_norm_training_val = MU_norm_training[choice == 1]
PHI_meas_training_train = PHI_meas_training[choice == 0]
PHI_meas_train_raw = PHI_meas_raw[key][choice == 0]
PHI_meas_std = np.std(PHI_meas_train_raw, axis=1)
PHI_meas_training_val = PHI_meas_training[choice == 1]
freq_training_train = freq[key][choice == 0]
freq_training_val = freq[key][choice == 1]
d_training_train = d[key][choice == 0]
d_training_val = d[key][choice == 1]
seq_len_train = seq_len[key][choice == 0]
seq_len_val = seq_len[key][choice == 1]
MUa_train = MUa_training[choice == 0]
MUa_val = MUa_training[choice == 1]
MUsp_train = MUsp_training[choice == 0]
MUsp_val = MUsp_training[choice == 1]
count_label_train = count_label[choice == 0]
count_label_val = count_label[choice == 1]
pos_data_train = pos_data[choice == 0]
pos_data_val = pos_data[choice == 1]
inc_areas_train = inc_areas[choice == 0]
inc_areas_val = inc_areas[choice == 1]
N_count_train = np.zeros_like(N_count)
N_count_val = np.zeros_like(N_count)
n = 0
for i in range(len(N_count)):
    ni = N_count[i]
    N_count_train[i] = np.where(choice[n:(n + ni)] == 0)[0].size
    N_count_val[i] = ni - N_count_train[i]
    n += ni
idx = np.lexsort((count_label_train, seq_len_train))[::-1]
index_array_dis = idx
index_array_gan = np.copy(idx)
ia = np.arange(train_size)
nsc_u, idx = np.unique((count_label_train, seq_len_train), axis=1, return_inverse=True)
nsc_u = nsc_u[:, ::-1]
idx = len(nsc_u) - 1 - idx
idx = idx[index_array_dis]
ns_u, idx2 = np.unique(seq_len_train, return_inverse=True)
idx2 = idx2[index_array_dis]
# choice = data_cat
batch_size = 32
prob_param = 20.0
prob_param2 = 1.0
# label_softness = 0.1
# metrics_eval_n = 50
# metrics_eval_step = 20
hard_valid = np.ones(val_size)
hard_fake = np.zeros(val_size)
# noise_max = 0.1
# noise_add_max = 1e-3
datagen = ImageDataGenerator(rotation_range=360, horizontal_flip=True, vertical_flip=True)


# soft_label = False
# augment = True
# min_img_loss_avg = np.inf
# min_dis_loss_avg = np.inf
# img_loss_baseline_factor = 1.1
# img_loss_threshold = 1.2
# img_loss_threshold2 = 0.6
# prob_augment = 0.5
# gan_d_acc = [0.5, 0.5]
# init_train = 100
apply_noise = False
print(gan.metrics_names)
print(discriminator.metrics_names)
# model_name = input('Enter model name: ')
model_name = 'model46log'
checkpoint = ModelCheckpoint(model_name + '_{epoch:02d}.h5', period=10)
checkpoint.set_model(gan)
checkpoint_gen = ModelCheckpoint(model_name + '_gen_{epoch:02d}.h5', period=10)
checkpoint_gen.set_model(generator)
checkpoint_dis = ModelCheckpoint(model_name + '_dis_{epoch:02d}.h5', period=10)
checkpoint_dis.set_model(discriminator)


def run_epoch(start, end=None):
    global PHI_meas_training_train

    if end is None:
        end = start
        start = 0

    history = History()
    logger = BaseLogger(stateful_metrics=['discriminator_' + s for s in discriminator.stateful_metric_names] +
                                         ['gan_' + s for s in gan.stateful_metric_names])
    time_history = TimeHistory()
    callbacks = [logger, time_history, history, checkpoint, checkpoint_gen, checkpoint_dis]
    out_labels = ['discriminator_' + s for s in discriminator.metrics_names] +\
                 ['discriminator_real_' + s for s in discriminator.metrics_names] +\
                 ['discriminator_fake_' + s for s in discriminator.metrics_names] +\
                 ['gan_' + s for s in gan.metrics_names]
    dis_metrics_names_len = len(discriminator.metrics_names)
    out_labels[dis_metrics_names_len*3 + 8] = 'gan_discriminator_pos_loss'
    out_labels[dis_metrics_names_len*3 + 9] = 'gan_discriminator_area_loss'
    callback_metrics = copy.copy(out_labels) + ['val_' + n for n in out_labels]
    callbacks = CallbackList(callbacks)
    # callbacks = [TimeHistory(), EarlyStopping(monitor='loss', min_delta=0.1, patience=100,
    #                                           restore_best_weights=True, verbose=1)]
    # early_stop = EarlyStopping(patience=200, verbose=1)
    callbacks.set_params({
        'batch_size': batch_size,
        'epochs': end,
        'steps': None,
        'samples': train_size,
        'verbose': 2,
        'do_validation': True,
        'metrics': callback_metrics,
    })
    callbacks.on_train_begin()
    # n = N_count_train[0]
    # np.random.shuffle(index_array_dis[:n])
    # np.random.shuffle(index_array_gan[:n])
    # for i in range(1, len(N_count_train) - 1):
    #     ni = N_count_train[i]
    #     np.random.shuffle(index_array_dis[n:(n + ni)])
    #     np.random.shuffle(index_array_gan[n:(n + ni)])
    #     n += ni
    # np.random.shuffle(index_array_dis[n:])
    # np.random.shuffle(index_array_gan[n:])
    for i in range(len(nsc_u)):
        np.random.shuffle(index_array_dis[idx == i])
        np.random.shuffle(index_array_gan[idx == i])
    for epoch in range(start, end):
        for m in discriminator.stateful_metric_functions:
            m.reset_states()
        for m in gan.stateful_metric_functions:
            m.reset_states()
        callbacks.on_epoch_begin(epoch)
        epoch_logs = {}
        progress_bar = None
        for u in ns_u:
            np.random.shuffle(ia[ns_u[idx2] == u])
        if apply_noise:
            PHI_meas_training_train = PHI_meas_training[choice == 0]
            noise_flag = np.random.random((train_size, 1)) >= 0.5
            meas_noise = np.random.random((train_size, 1)) * noise_max * noise_flag
            meas_add_noise = np.random.random((train_size, 1)) * noise_add_max * noise_flag
            noise = np.apply_along_axis(lambda a: (np.random.normal(scale=1 / np.sqrt(2),
                                                                    size=PHI_meas_train_raw.shape[1:]) +
                                                   np.random.normal(scale=1 / np.sqrt(2),
                                                                    size=PHI_meas_train_raw.shape[1:]) * 1j) * a, 1,
                                        meas_noise)
            raw = PHI_meas_train_raw * (1 + noise)
            noise = np.apply_along_axis(lambda a: (np.random.normal(scale=1 / np.sqrt(2),
                                                                    size=PHI_meas_train_raw.shape[1:]) +
                                                   np.random.normal(scale=1 / np.sqrt(2),
                                                                    size=PHI_meas_train_raw.shape[1:]) * 1j) * a, 1,
                                        meas_add_noise)
            raw = raw + np.std(raw, axis=(1, 2)).reshape((-1, 1, 1)) * noise
            PHI_meas_training_train[:, :, :(ns_target - 1) * 2] = raw.view('float64')
        num_batches = (train_size + batch_size - 1) // batch_size
        # if epoch > 0 and epoch % metrics_eval_step == 0:
        # hist_avg = {}
        # for k, v in history.history.items():
        #     if k.startswith('val_'):
        #         l = k[4:]
        #     else:
        #         l = k
        #     if l in logger.stateful_metrics:
        #         hist_avg[k] = v[-1]
        #     else:
        #         hist_avg[k] = np.average(v[-metrics_eval_n:])
        # img_loss_avg = hist_avg['val_gan_' + gan.metrics_names[0]] - hist_avg['val_gan_' + gan.metrics_names[7]]
        # dis_loss_avg = hist_avg['val_discriminator_' + discriminator.metrics_names[0]]
        # min_img_loss = np.min(history.history['val_gan_' + gan.metrics_names[0]]) \
        #                - np.min(history.history['val_gan_' + gan.metrics_names[7]])
        # if img_loss_avg > min_img_loss_avg * img_loss_baseline_factor:
        #     if not augment:
        #         augment = True
        #     elif img_loss_avg >= img_loss_threshold and min_img_loss >= img_loss_threshold2:
        #         soft_label = True
        # elif dis_loss_avg > min_dis_loss_avg:
        #     if augment:
        #         augment = False
        #     elif img_loss_avg < img_loss_threshold:
        #         soft_label = False
        # img_loss_avg = np.average(history.history['val_gan_' + gan.metrics_names[0]][-metrics_eval_n:])\
        #                - np.average(history.history['val_gan_' + gan.metrics_names[7]][-metrics_eval_n:])
        # if soft_label and img_loss_avg < img_loss_threshold:
        #     soft_label = False
        # if np.random.random() > 0.5:
        #     augment = not augment
        # min_img_loss_avg = min(img_loss_avg, min_img_loss_avg)
        # min_dis_loss_avg = min(dis_loss_avg, min_dis_loss_avg)
        # if epoch >= init_train:
        #     if gan_d_acc[0] < 0.5 or gan_d_acc[1] < 0.5:
        #         prob_augment = 0.8
        #     else:
        #         prob_augment = 0.5
        #     if augment == (np.random.random() < prob_augment):
        #         augment = not augment
        batch_end = 0
        # prob_augment = np.random.random()
        # if dump:
        #     mean_freq = np.zeros(num_batches)
        #     std_freq = np.zeros(num_batches)
        #     mean_d = np.zeros(num_batches)
        #     std_d = np.zeros(num_batches)
        #     mean_ns = np.zeros(num_batches)
        #     std_ns = np.zeros(num_batches)
        #     MUa_loss = np.zeros(num_batches)
        #     MUsp_loss = np.zeros(num_batches)
        #     augment_arr = np.zeros(num_batches)
        #     seq_sz_arr = np.zeros(num_batches)
        # for batch_index, batch_MU_training_aug in enumerate(datagen.flow(MU_training_train[index_array_dis[ia]],
        #                                                                  batch_size=32, shuffle=False)):
        for batch_index in range(num_batches):
            # if batch_index >= num_batches:
            #     break
            augment = np.random.random() < 0.4
            batch_start = batch_end
            batch_end = min(train_size, batch_end + batch_size)
            batch_ids = index_array_dis[ia[batch_start:batch_end]]
            batch_seq_len = seq_len_train[batch_ids]
            # if dump:
            #     augment_arr[batch_index] = augment
            #     mean_freq[batch_index] = np.mean(batch_freq)
            #     std_freq[batch_index] = np.std(batch_freq)
            #     mean_d[batch_index] = np.mean(batch_d)84+1
            #     std_d[batch_index] = np.std(batch_d)
            #     mean_ns[batch_index] = np.mean(batch_seq_len)
            #     std_ns[batch_index] = np.std(batch_seq_len)
            #     seq_sz_arr[batch_index] = seq_sz
            size = len(batch_ids)
            half2 = len(batch_ids)//2
            half = len(batch_ids) - half2
            batch_logs = {'batch': batch_index, 'size': size}
            callbacks.on_batch_begin(batch_index, batch_logs)
            MU_gt = [MU_training_train[batch_ids, 0], MU_training_train[batch_ids, 1]]
            MU_gt = [im.reshape(im.shape[:1] + (1,) + im.shape[1:]) for im in MU_gt]
            # sel_prob = np.random.random() >= 0.5
            max_seq_len = max(batch_seq_len)
            mask = np.random.random((max_seq_len, 1)) >= 0.1
            # seq_sel = np.zeros(PHI_meas_training_train.shape[1], dtype='uint8')
            # if sel_prob:
            #     seq_sel[:max_seq_len] = np.random.random(max_seq_len) >= 0.1
            # else:
            #     seq_sel[:max_seq_len] = 1
            # in_gen = [PHI_meas_training_train[batch_ids][:, seq_sel] * mask[seq_sel], freq_training_train[batch_ids],
            #           d_training_train[batch_ids]]
            in_gen = [PHI_meas_training_train[batch_ids][:, :max_seq_len]  * mask, freq_training_train[batch_ids],
                      d_training_train[batch_ids]]
            # batch_pos_data = pos_data_train[batch_ids]
            # batch_pos_data2 = np.clip(batch_pos_data + truncnorm.rvs(-2, 2, scale=0.25, size=batch_pos_data.shape),
            #                           0, 1)
            # batch_inc_areas = inc_areas_train[batch_ids]
            # batch_inc_areas2 = batch_inc_areas + batch_inc_areas * truncnorm.rvs(-2, 2, scale=0.25,
            #                                                                      size=batch_inc_areas.shape)
            if augment:
                noise_param = np.random.random(half2)
                blur_param = np.random.random(half2)
                noise_param *= blur_param
                rotate_param = np.random.random(half2) * 360
                # MU_gt_aug = [np.concatenate([MU_gt[i][:half], batch_MU_training_aug[half:, [i]]]) for i in range(2)]
                MU_gt_aug = [np.copy(im) for im in MU_gt]
                # batch_pos_data = batch_pos_data.copy()
                # batch_pos_data[half:] = transform_theta(batch_pos_data[half:], rotate_param.reshape((-1, 1)))
                for i, bi in enumerate(batch_ids[half:]):
                    temp = datagen.apply_transform(MU_gt_aug[0][half+i], {'theta': rotate_param[i]})[0]
                    temp = mask_image(temp, MUa_train[bi])
                    # temp = temp + temp * np.random.normal(scale=noise_param[i] * 0.5, size=temp.shape)
                    temp = temp + temp * truncnorm.rvs(-2, 2, scale=noise_param[i] * 0.5, size=temp.shape)
                    temp = gaussian_filter(temp, blur_param[i] * 5.0)
                    MU_gt_aug[0][half+i, 0] = mask_image(temp, 0.0)
                    temp = datagen.apply_transform(MU_gt_aug[1][half+i], {'theta': rotate_param[i]})[0]
                    temp = mask_image(temp, MUsp_train[bi])
                    # temp = temp + temp * np.random.normal(scale=noise_param[i] * 0.5, size=temp.shape)
                    temp = temp + temp * truncnorm.rvs(-2, 2, scale=0.5, size=temp.shape)
                    temp = gaussian_filter(temp, blur_param[i] * 5.0)
                    MU_gt_aug[1][half+i, 0] = mask_image(temp, 0.0)
                # temp = np.clip(blur_param, prob_param2 / prob_param, 1.0 - prob_param2 / prob_param)
                valid = np.concatenate([hard_valid[:half] - 0.05, 0.95 - 0.05 * np.random.random(half2)])
                # valid = 1.0 - label_softness * np.random.beta(temp * prob_param, (1.0 - temp) * prob_param)
                # if soft_label:
                #     fake = label_softness * np.random.random(len(batch_ids))
                #     valid = 1.0 - label_softness * np.random.beta(temp * prob_param, (1.0 - temp) * prob_param)
                # else:
                #     fake = hard_fake[:len(batch_ids)]
                #     valid = hard_valid[:len(batch_ids)] - label_softness
                # d_loss1 = discriminator.train_on_batch(
                #     MU_gt_aug, [valid, batch_pos_data, batch_inc_areas], sample_weight=
                #     [np.concatenate([np.ones(half) * 1.2, np.ones(half2) * (size - half * 1.2) / half2]),
                #      np.ones(size) * 1.9, np.ones(size) * 1.9])
                d_loss1 = discriminator.train_on_batch(
                    MU_gt_aug, valid, sample_weight=
                    np.concatenate([np.ones(half) * 1.2, np.ones(half2) * (size - half * 1.2) / half2]))
            else:
                MU_gt_aug = MU_gt
                valid = hard_valid[:size] - 0.05
                # valid = 1.0 - 0.05 * np.random.random(size)
                # valid = 1.0 - label_softness * np.random.random(len(batch_ids))
                # valid = 1.0 - label_softness * np.random.beta(prob_param2,
                #                                               prob_param - prob_param2, size=len(batch_ids))
                # if soft_label:
                #     fake = label_softness * np.random.random(len(batch_ids))
                #     valid = 1.0 - label_softness * np.random.beta(
                #         prob_param2, prob_param - prob_param2, size=len(batch_ids))
                # else:
                #     fake = hard_fake[:len(batch_ids)]
                #     valid = hard_valid[:len(batch_ids)] - label_softness
                # d_loss1 = discriminator.train_on_batch(MU_gt_aug, [valid, batch_pos_data, batch_inc_areas],
                #                                        sample_weight=[np.ones(size), np.ones(size) * 1.9,
                #                                                       np.ones(size) * 1.9])
                d_loss1 = discriminator.train_on_batch(MU_gt_aug, valid)
            # fake = hard_fake[:size]
            fake = 0.05 * np.random.random(size)
            # if epoch < init_train:
            #     valid -= label_softness * np.random.random(len(batch_ids)) * (init_train - epoch) / init_train
            #     fake += label_softness * np.random.random(len(batch_ids)) * (init_train - epoch) / init_train
            # d_loss1 = discriminator.train_on_batch([np.concatenate([im1, im2], axis=0)
            #                                        for im1, im2 in zip(MU_gt_aug, MU_gt)],
            #                                        np.concatenate([valid, hard_valid[:len(batch_ids)]], axis=0),
            #                                        sample_weight=np.ones(2 * len(batch_ids)) * 0.5)
            # d_loss2 = discriminator.train_on_batch(generator.predict(in_gen)[:2],
            #                                        [fake, batch_pos_data2, batch_inc_areas2],
            #                                        sample_weight=[np.ones(size), np.ones(size) * 0.1,
            #                                                       np.ones(size) * 0.1])
            d_loss2 = discriminator.train_on_batch(generator.predict(in_gen)[:2], fake)
            # d_loss = discriminator.train_on_batch([np.concatenate([im1, im2], axis=0)
            #                                        for im1, im2 in zip(MU_gt_aug, generator.predict(in_gen)[:2])],
            #                                       np.concatenate([valid, fake], axis=0))
            d_loss = np.zeros(len(discriminator.metrics_names))
            for i, l in enumerate(discriminator.metrics_names):
                if l in discriminator.stateful_metric_names:
                    d_loss[i] = d_loss2[i]
                else:
                    d_loss[i] = 0.5 * (d_loss1[i] + d_loss2[i])
            # for l in discriminator.layers:
            #     weights = l.get_weights()
            #     weights = [np.clip(w, -clip_value, clip_value) for w in weights]
            #     l.set_weights(weights)
            batch_ids = index_array_gan[ia[batch_start:batch_end]]
            MU_gt = [MU_training_train[batch_ids, 0], MU_training_train[batch_ids, 1]]
            MU_gt = [im.reshape(im.shape[:1] + (1,) + im.shape[1:]) for im in MU_gt]
            # in_gen = [PHI_meas_training_train[batch_ids][:, seq_sel] * mask[seq_sel], freq_training_train[batch_ids],
            #           d_training_train[batch_ids]]
            in_gen = [PHI_meas_training_train[batch_ids], freq_training_train[batch_ids],
                      d_training_train[batch_ids]]
            # batch_pos_data = pos_data_train[batch_ids]
            # batch_inc_areas = inc_areas_train[batch_ids]
            MU_norm_gt = [MU_norm_training_train[batch_ids, 0], MU_norm_training_train[batch_ids, 1]]
            MU_norm_gt = [im.reshape(im.shape[:1] + (1,) + im.shape[1:]) for im in MU_norm_gt]
            # g_loss = gan.train_on_batch(in_gen, [MU_gt[0], MU_gt[1], MU_norm_gt[0], MU_norm_gt[1],
            #                                      MUa_train[batch_ids], MUsp_train[batch_ids],
            #                                      hard_valid[:size], batch_pos_data, batch_inc_areas])
            g_loss = gan.train_on_batch(in_gen, [MU_gt[0], MU_gt[1], MU_norm_gt[0], MU_norm_gt[1],
                                                 MUa_train[batch_ids], MUsp_train[batch_ids], hard_valid[:size]])
            for l, o in zip(out_labels, np.concatenate([d_loss, d_loss1, d_loss2, g_loss])):
                batch_logs[l] = o
            callbacks.on_batch_end(batch_index, batch_logs)
            if progress_bar is None:
                progress_bar = tqdm(total=train_size, desc='Epoch {}/{}'.format(epoch + 1, end), unit='samples')
            # if dump:
            #     MUa_loss[batch_index] = g_loss[3]
            #     MUsp_loss[batch_index] = g_loss[4]
            #     progress_bar.set_postfix(train_on_batch=('D loss: dis [r:%g f:%g] aux [r:%g], acc.: dis [r:%.2f%% ' +
            #                                              'f:%.2f%%] | [G total_loss: %g, img_loss: %g, d_loss: dis' +
            #                                              ' %g aux %g, d_acc.: dis %.2f%% | MUa_contrast_loss:%g,' +
            #                                              ' MUsp_contrast_loss:%g | stats: freq [mean:%g std:%g]' +
            #                                              ' d [mean:%g std:%g] ns [mean:%g std:%g] size %g') %
            #                                             (d_loss1[1], d_loss2[1], d_loss1[2], 100 * d_loss1[3],
            #                                              100 * d_loss2[3], g_loss[0], g_loss[0] - g_loss[7]*dis_lw[0] -
            #                                              g_loss[8]*dis_lw[1], g_loss[7], g_loss[8],
            #                                              100 * g_loss[-2*dis_metrics_len], g_loss[3], g_loss[4],
            #                                              mean_freq[batch_index], std_freq[batch_index],
            #                                              mean_d[batch_index], std_d[batch_index],
            #                                              mean_ns[batch_index], std_ns[batch_index], seq_sz),
            #                              augment=augment)
            # else:
            # progress_bar.set_postfix(train_on_batch=('D loss: dis [r:%g f:%g] pos [r:%g] area [r:%g], acc.: dis ' +
            #                                          '[r:%.2f%% f:%.2f%%] | [G total_loss: %g, img_loss: %g, d_loss:' +
            #                                          ' dis %g pos %g area %g, d_acc.: dis %.2f%% | MUa_contrast_' +
            #                                          'loss:%g, MUsp_contrast_loss:%g') %
            #                                         (d_loss1[1], d_loss2[1], d_loss1[2], d_loss1[3], 100 * d_loss1[4],
            #                                          100 * d_loss2[4], g_loss[0], g_loss[0] - g_loss[7]*dis_lw[0] -
            #                                          g_loss[8]*dis_lw[1] - g_loss[9]*dis_lw[2], g_loss[7], g_loss[8],
            #                                          g_loss[9], 100 * g_loss[-2*dis_metrics_len], g_loss[3], g_loss[4]),
            #                          augment=augment)
            progress_bar.set_postfix(train_on_batch=('D loss: dis [r:%g f:%g], acc.: dis ' +
                                                     '[r:%.2f%% f:%.2f%%] | [G total_loss: %g, img_loss: %g, d_loss:' +
                                                     ' dis %g, d_acc.: dis %.2f%% | MUa_contrast_' +
                                                     'loss:%g, MUsp_contrast_loss:%g') %
                                                    (d_loss1[0], d_loss2[0], 100 * d_loss1[1],
                                                     100 * d_loss2[1], g_loss[0], g_loss[0] - g_loss[7]*dis_lw[0],
                                                     g_loss[7], 100 * g_loss[-1], g_loss[3], g_loss[4]),
                                     augment=augment)
            progress_bar.update(size)
        MU_gt = [MU_training_val[:, np.array([0])], MU_training_val[:, np.array([1])]]
        # d_val_loss1 = discriminator.evaluate(MU_gt, [hard_valid, pos_data_val, inc_areas_val],
        #                                      batch_size=batch_size, verbose=0)
        # d_val_loss2 = discriminator.evaluate(generator.predict(
        #     [PHI_meas_training_val, freq_training_val, d_training_val])[:2], [hard_fake, pos_data_val, inc_areas_val],
        #                                      batch_size=batch_size, verbose=0)
        d_val_loss1 = discriminator.evaluate(MU_gt, hard_valid, batch_size=batch_size, verbose=0)
        d_val_loss2 = discriminator.evaluate(generator.predict(
            [PHI_meas_training_val, freq_training_val, d_training_val])[:2], hard_fake, batch_size=batch_size,
                                             verbose=0)
        # d_val_loss = discriminator.evaluate([np.concatenate([im1, im2], axis=0)
        #                                      for im1, im2 in zip(MU_gt,
        #                                                          generator.predict(PHI_meas_training_val)[:2])],
        #                                     np.concatenate([hard_valid, hard_fake], axis=0), verbose=0)
        d_val_loss = np.zeros(len(discriminator.metrics_names))
        for i, l in enumerate(discriminator.metrics_names):
            if l in discriminator.stateful_metric_names:
                d_val_loss[i] = d_val_loss2[i]
            else:
                d_val_loss[i] = 0.5 * (d_val_loss1[i] + d_val_loss2[i])
        MU_norm_gt = [MU_norm_training_val[:, np.array([0])], MU_norm_training_val[:, np.array([1])]]
        # progress_bar.set_postfix(validate_on_epoch=('D loss: dis [r:%g f:%g] pos [r:%g f:%g] area [r:%g f:%g], ' +
        #                                             'acc.: dis [r:%.2f%% f:%.2f%%] validating generator..') %
        #                                            (d_val_loss1[1], d_val_loss2[1], d_val_loss1[2], d_val_loss2[2],
        #                                             d_val_loss1[3], d_val_loss2[3], 100 * d_val_loss1[4],
        #                                             100 * d_val_loss2[4]))
        progress_bar.set_postfix(validate_on_epoch=('D loss: dis [r:%g f:%g], ' +
                                                    'acc.: dis [r:%.2f%% f:%.2f%%] validating generator..') %
                                                   (d_val_loss1[0], d_val_loss2[0], 100 * d_val_loss1[1],
                                                    100 * d_val_loss2[1]))
        # g_val_loss = gan.evaluate([PHI_meas_training_val, freq_training_val, d_training_val],
        #                           [MU_gt[0], MU_gt[1], MU_norm_gt[0], MU_norm_gt[1], MUa_val, MUsp_val, hard_valid,
        #                            pos_data_val, inc_areas_val], batch_size=batch_size, verbose=0)
        g_val_loss = gan.evaluate([PHI_meas_training_val, freq_training_val, d_training_val],
                                  [MU_gt[0], MU_gt[1], MU_norm_gt[0], MU_norm_gt[1], MUa_val, MUsp_val, hard_valid],
                                  batch_size=batch_size, verbose=0)
        for l, o in zip(out_labels, np.concatenate([d_val_loss, d_val_loss1, d_val_loss2, g_val_loss])):
            epoch_logs['val_' + l] = o
        # num_batches = (val_size + batch_size - 1) // batch_size
        # for batch_index in range(num_batches):
        #     batch_start = batch_index * batch_size
        #     batch_end = min(val_size, (batch_index + 1) * batch_size)
        #     MU_gt = [MU_training_val[batch_start:batch_end, 0], MU_training_val[batch_start:batch_end, 0]]
        #     MU_gt = [im.reshape(im.shape[:1] + (1,) + im.shape[1:]) for im in MU_gt]
        #     d_loss1 = discriminator.test_on_batch(MU_gt, hard_valid[:(batch_end - batch_start)])
        #     in_gen = PHI_meas_training_val[batch_start:batch_end]
        #     d_loss2 = discriminator.test_on_batch(generator.predict(in_gen)[:2], hard_fake[:(batch_end - batch_start)])
        #     d_loss = np.zeros(len(discriminator.metrics_names))
        #     for i, l in enumerate(discriminator.metrics_names):
        #         if l in discriminator.stateful_metric_names:
        #             d_loss[i] = d_loss2[i]
        #         else:
        #             d_loss[i] = 0.5 * (d_loss1[i] + d_loss2[i])
        #     MU_norm_gt = [MU_norm_training_val[batch_start:batch_end, 0],
        #                   MU_norm_training_val[batch_start:batch_end, 1]]
        #     MU_norm_gt = [im.reshape(im.shape[:1] + (1,) + im.shape[1:]) for im in MU_norm_gt]
        #     g_loss = gan.test_on_batch(in_gen, [MU_gt[0], MU_gt[1], MU_norm_gt[0], MU_norm_gt[1],
        #                                       MUa_val[batch_start:batch_end], MUsp_val[batch_start:batch_end],
        #                                       hard_valid[:(batch_end - batch_start)]])
        #     for l, o in zip(out_labels, np.concatenate([d_loss, g_loss])):
        #         if l in logger.stateful_metrics:
        #             epoch_logs['val_' + l] = o
        #         else:
        #             if 'val_' + l in epoch_logs:
        #                 epoch_logs['val_' + l] += o * (batch_end - batch_start)
        #             else:
        #                 epoch_logs['val_' + l] = o * (batch_end - batch_start)
        #     pbar.set_postfix(validate_on_batch=('[D loss: %g, acc.: %.2f%%] [G total_loss: %g, img_loss: %g, ' +
        #                                         'd_loss: %g, d_acc.: %.2f%%]') %
        #                                        (d_loss[0], 100 * d_loss[1], g_loss[0], g_loss[0] - g_loss[7],
        #                                         g_loss[7], 100 * g_loss[-1]), soft_label=soft_label, augment=augment)
        #     pbar.update(batch_end - batch_start)
        # for l in out_labels:
        #     if l not in logger.stateful_metrics:
        #         epoch_logs['val_' + l] /= val_size
        callbacks.on_epoch_end(epoch, epoch_logs)
        # gan_d_acc[0] = epoch_logs[out_labels[-1]]
        # gan_d_acc[1] = epoch_logs['val_' + out_labels[-1]]
        # progress_bar.set_postfix(train=('D loss: dis [r:%g f:%g] pos [r:%g] area [r:%g], acc.: dis [r:%.2f%% ' +
        #                                 'f:%.2f%%] | [G total_loss: %g, img_loss: %g, d_loss: dis %g pos %g area %g,' +
        #                                 ' d_acc.: dis %.2f%% | MUa_contrast_loss:%g, MUsp_contrast_loss:%g') %
        #                                (epoch_logs[out_labels[dis_metrics_names_len + 1]],
        #                                 epoch_logs[out_labels[dis_metrics_names_len*2 + 1]],
        #                                 epoch_logs[out_labels[dis_metrics_names_len + 2]],
        #                                 epoch_logs[out_labels[dis_metrics_names_len + 3]],
        #                                 100 * epoch_logs[out_labels[dis_metrics_names_len + 4]],
        #                                 100 * epoch_logs[out_labels[dis_metrics_names_len*2 + 4]],
        #                                 epoch_logs[out_labels[dis_metrics_names_len*3]],
        #                                 epoch_logs[out_labels[dis_metrics_names_len*3]] -
        #                                 epoch_logs[out_labels[dis_metrics_names_len*3 + 7]]*dis_lw[0] -
        #                                 epoch_logs[out_labels[dis_metrics_names_len*3 + 8]]*dis_lw[1] -
        #                                 epoch_logs[out_labels[dis_metrics_names_len*3 + 9]]*dis_lw[2],
        #                                 epoch_logs[out_labels[dis_metrics_names_len*3 + 7]],
        #                                 epoch_logs[out_labels[dis_metrics_names_len*3 + 8]],
        #                                 epoch_logs[out_labels[dis_metrics_names_len*3 + 9]],
        #                                 100 * epoch_logs[out_labels[-2*dis_metrics_len]],
        #                                 epoch_logs[out_labels[dis_metrics_names_len*3 + 3]],
        #                                 epoch_logs[out_labels[dis_metrics_names_len*3 + 4]]),
        #                          validation=('D loss: dis [r:%g f:%g] pos [r:%g f:%g] area [r:%g f:%g], acc.: dis ' +
        #                                      '[r:%.2f%% f:%.2f%%] | [G total_loss: %g, img_loss: %g, d_loss: dis ' +
        #                                      '%g pos %g area %g, d_acc.: dis %.2f%% | MUa_contrast_loss:%g, ' +
        #                                      'MUsp_contrast_loss:%g') %
        #                                     (epoch_logs['val_' + out_labels[dis_metrics_names_len + 1]],
        #                                      epoch_logs['val_' + out_labels[dis_metrics_names_len*2 + 1]],
        #                                      epoch_logs['val_' + out_labels[dis_metrics_names_len + 2]],
        #                                      epoch_logs['val_' + out_labels[dis_metrics_names_len*2 + 2]],
        #                                      epoch_logs['val_' + out_labels[dis_metrics_names_len + 3]],
        #                                      epoch_logs['val_' + out_labels[dis_metrics_names_len*2 + 3]],
        #                                      100 * epoch_logs['val_' + out_labels[dis_metrics_names_len + 4]],
        #                                      100 * epoch_logs['val_' + out_labels[dis_metrics_names_len*2 + 4]],
        #                                      epoch_logs['val_' + out_labels[dis_metrics_names_len*3]],
        #                                      epoch_logs['val_' + out_labels[dis_metrics_names_len*3]] -
        #                                      epoch_logs['val_' + out_labels[dis_metrics_names_len*3 + 7]]*dis_lw[0] -
        #                                      epoch_logs['val_' + out_labels[dis_metrics_names_len*3 + 8]]*dis_lw[1] -
        #                                      epoch_logs['val_' + out_labels[dis_metrics_names_len*3 + 9]]*dis_lw[2],
        #                                      epoch_logs['val_' + out_labels[dis_metrics_names_len*3 + 7]],
        #                                      epoch_logs['val_' + out_labels[dis_metrics_names_len*3 + 8]],
        #                                      epoch_logs['val_' + out_labels[dis_metrics_names_len*3 + 9]],
        #                                      100 * epoch_logs['val_' + out_labels[-2*dis_metrics_len]],
        #                                      epoch_logs['val_' + out_labels[dis_metrics_names_len*3 + 3]],
        #                                      epoch_logs['val_' + out_labels[dis_metrics_names_len*3 + 4]]))
        progress_bar.set_postfix(train=('D loss: dis [r:%g f:%g], acc.: dis [r:%.2f%% ' +
                                        'f:%.2f%%] | [G total_loss: %g, img_loss: %g, d_loss: dis %g,' +
                                        ' d_acc.: dis %.2f%% | MUa_contrast_loss:%g, MUsp_contrast_loss:%g') %
                                       (epoch_logs[out_labels[dis_metrics_names_len]],
                                        epoch_logs[out_labels[dis_metrics_names_len*2]],
                                        100 * epoch_logs[out_labels[dis_metrics_names_len + 1]],
                                        100 * epoch_logs[out_labels[dis_metrics_names_len*2 + 1]],
                                        epoch_logs[out_labels[dis_metrics_names_len*3]],
                                        epoch_logs[out_labels[dis_metrics_names_len*3]] -
                                        epoch_logs[out_labels[dis_metrics_names_len*3 + 7]]*dis_lw[0],
                                        epoch_logs[out_labels[dis_metrics_names_len*3 + 7]],
                                        100 * epoch_logs[out_labels[-1]],
                                        epoch_logs[out_labels[dis_metrics_names_len*3 + 3]],
                                        epoch_logs[out_labels[dis_metrics_names_len*3 + 4]]),
                                 validation=('D loss: dis [r:%g f:%g], acc.: dis ' +
                                             '[r:%.2f%% f:%.2f%%] | [G total_loss: %g, img_loss: %g, d_loss: dis ' +
                                             '%g, d_acc.: dis %.2f%% | MUa_contrast_loss:%g, ' +
                                             'MUsp_contrast_loss:%g') %
                                            (epoch_logs['val_' + out_labels[dis_metrics_names_len]],
                                             epoch_logs['val_' + out_labels[dis_metrics_names_len*2]],
                                             100 * epoch_logs['val_' + out_labels[dis_metrics_names_len + 1]],
                                             100 * epoch_logs['val_' + out_labels[dis_metrics_names_len*2 + 1]],
                                             epoch_logs['val_' + out_labels[dis_metrics_names_len*3]],
                                             epoch_logs['val_' + out_labels[dis_metrics_names_len*3]] -
                                             epoch_logs['val_' + out_labels[dis_metrics_names_len*3 + 7]]*dis_lw[0],
                                             epoch_logs['val_' + out_labels[dis_metrics_names_len*3 + 7]],
                                             100 * epoch_logs['val_' + out_labels[-1]],
                                             epoch_logs['val_' + out_labels[dis_metrics_names_len*3 + 3]],
                                             epoch_logs['val_' + out_labels[dis_metrics_names_len*3 + 4]]))
        progress_bar.close()
    callbacks.on_train_end()
    hist_data = {
        'train_start': time_history.train_time_start,
        'train_time': time_history.train_time,
        'epoch_time': time_history.times,
        'history': history.history,
        'choice': np.where(choice == 1)[0].tolist()
    }
    # if dump:
    #     return hist_data, (MUa_loss, MUsp_loss, mean_freq, std_freq, mean_d, std_d, mean_ns, std_ns, augment_arr,
    #                        seq_sz_arr)
    # else:
    return hist_data


hist_data = run_epoch(100)


Using TensorFlow backend.





Loading datasets..
MAT files loaded.
Create dictionaries..
Dictionaries created.
Prepare measurement data..
All datasets loaded.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






Generator model summary:
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 7)      0                                            
__________________________________________________________________________________________________
masking_1 (Masking)             (None, None, 7)      0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
______________________________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
Epoch 1/100:   0%|          | 0/8000 [00:00<?, ?samples/s, augment=1, train_on_batch=D loss: dis [r:1.2131 f:0.459484], acc.: dis [r:12.50% f:90.62%] | [G total_loss: 19.1345, img_loss: 17.8818, d_loss: dis 1.25269, d_acc.: dis 15.62% | MUa_contrast_loss:1.4023, MUsp_contrast_loss:1.46524]/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
Epoch 29/100:  34%|███▍      | 2720/8000 [01:42<03:29, 25.20samples/s, augment=1, train_on_batch=D loss: dis [r:0.424685 f:0.191287]

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-4d1c930cd637>", line 645, in <module>
    hist_data = run_epoch(100)
  File "<ipython-input-2-4d1c930cd637>", line 353, in run_epoch
    temp = temp + temp * truncnorm.rvs(-2, 2, scale=noise_param[i] * 0.5, size=temp.shape)
  File "/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py", line 980, in rvs
    vals = self._rvs(*args)
  File "/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py", line 913, in _rvs
    Y = self._ppf(U, *args)
  File "/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py", line 7163, in _ppf
    return _truncnorm_ppf(q, a, b)
  File "/usr/local/lib/python3.6/dist-packages/scipy/stats/_continuous_distns.py", line 6933, in vf_wrapper
    return vf(*args)
  File "/usr/local/lib/pytho

KeyboardInterrupt: ignored